# Similarity Search Benchmarking

These benchmarks were originally run on an early 2015 MacBook Pro with a 2.7 GHz dual-core i5 processor and 8GB of memory. 

They make use of a ChEMBL_27 dataset. 
## Setup Work
### Imports

In [1]:
import mongordkit
import time
import pymongo
import rdkit
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from os import sys
import pandas as pd
from rdkit import Chem
from statistics import mean, median
import mongomock
from rdkit.Chem import AllChem
from mongordkit.Database import write
from mongordkit.Search import similarity
from mongordkit import Search

### Database Setup
Here we set up a database called `test` that will hold our molecules. We will construct a collection called `molecules_100K` to hold the first 100,000 molecules in the ChEMBL_27 dataset and a collection called `molecules_1M` to hold the first 1,000,000 molecules in the ChEMBL_27 dataset. If you have already run benchmarks from `mongo-rdkit` on your local MongoDB instance, these should have been set up already.

In [ ]:
# Initialize the client that will connect to the database.
client = pymongo.MongoClient()
db = client.test

In [ ]:
# If necessary, write the first 100,000 compounds to molecules_100K.
if db.molecules_100K.count_documents({}) != 100000:
    write.WriteFromSDF(db.molecules_100K, '../../../chembl_27.sdf', chunk_size=1000, limit=100000)

In [ ]:
# If necessary, write the first 1,000,000 compounds to molecules_1M.
if db.molecules_1M.count_documents({}) != 1000000:
    write.writeFromSDF(db.molecules_1M, '../../../chembl_27.sdf', chunk_size=1000, limit=1000000)

In [ ]:
# Let's ensure that there are actually 100,000 and 1M documents in these collections, respectively.
print(f"In molecules_100K: {db.molecules_100K.count_documents({})} documents")
print(f"In molecules_1M: {db.molecules_1M.count_documents({})} documents")

In [ ]:
# Next, we have to prepare the database for search by adding in fingerprints and hash collections.
Search.PrepareForSearch(db, db.molecules_100K, db.molecules_100KCt, db.molecules_100KPm)
Search.PrepareForSearch(db, db.molecules_1M, db.molecules_1MCt, db.molecules_1MPm)

### Query Set Setup
To benchmark, we'll use the first 200 compounds in ChEMBL. Let's get an rdmol for each of these and write them into a list. 

In [ ]:
first_200 = []
for rdmol in Chem.ForwardMolSupplier('../../data/test_data/first_200.props.sdf'): 
    first_200.append(rdmol)

## Benchmarks

We will search each compound five times against the target database, taking the mean value as representative of that molecule. We'll then take the median and mean for all 200 compounds, repeating the entire process for thresholds 0.7, 0.75, 0.8, 0.85, and 0.9. 

We will benchmark both the `SimSearchAggregate` and `SimSearchLSH` methods, keeping in mind that the LSH method does not return exact results. 

In [ ]:
thresholds = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
repetitions = 5

### `SimSearchAggregate`

In [ ]:
# Benchmark against the first 100,000 molecules in ChEMBL. 
aggregate_means_100K = []
aggregate_medians_100K = []

for t in thresholds: 
    print(f"Measuring performance for similarity threshold {t}...")
    query_times = []
    for rdmol in first_200:
        temp_times = []
        for r in range(repetitions):
            start = time.time()
            _ = similarity.SimSearchAggregate(rdmol, db.molecules_100K, db.molecules_100KCt, threshold=t)
            end = time.time()
            temp_times.append(end - start)
        query_times.append(mean(temp_times))
    aggregate_means_100K.append([t, mean(query_times)])
    aggregate_medians_100K.append([t, median(query_times)])

print(f"Aggregate means: {aggregate_means_100K}")
print(f"Aggregate medians: {aggregate_medians_100K}")

Before we take a look at the 1M molecule dataset, let's graph these times to get a better idea of how similarity search increases in time required with lowered similarity thresholds: 

In [ ]:
x_list = [v[0] for v in aggregate_medians_100K]
y_list = [v[1] for v in aggregate_medians_100K]
plt.xlabel('thresholds')
plt.ylabel('time (s)')
plt.title('SimSearchAggregate medians / 100K dataset')
plt.plot(x_list, y_list)

And here are the equivalent benchmarks against a million-molecule dataset:

In [ ]:
# Benchmark against the first 1M molecules in ChEMBL. 
aggregate_means_1M = []
aggregate_medians_1M = []

for t in thresholds: 
    print(f"Measuring performance for similarity threshold {t}...")
    query_times = []
    for rdmol in first_200:
        temp_times = []
        for r in range(repetitions):
            start = time.time()
            _ = similarity.SimSearchAggregate(rdmol, db.molecules_1M, db.molecules_1MCt, threshold=t)
            end = time.time()
            temp_times.append(end - start)
        query_times.append(mean(temp_times))
    aggregate_means_1M.append([t, mean(query_times)])
    aggregate_medians_1M.append([t, median(query_times)])

print(f"Aggregate means: {aggregate_means_1M}")
print(f"Aggregate medians: {aggregate_medians_1M}")

In [ ]:
x_list = [v[0] for v in aggregate_medians_1M]
y_list = [v[1] for v in aggregate_medians_1M]
plt.xlabel('thresholds')
plt.ylabel('time (s)')
plt.title('SimSearchAggregate medians / 1M dataset')
plt.plot(x_list, y_list)

### `SimSearchLSH`

We will benchmark speed for LSH in the same way as we did for normal similarity search. As noted by the original ChEMBL authors of this approach, however, LSH also introduces an element of inaccuracy. Thus, we will also include a section on comparing results of `SimSearchAggregate` and `SimSearchLSH`.

In [ ]:
# Benchmark against the first 100,000 molecules in ChEMBL. 
LSH_means_100K = []
LSH_medians_100K = []

for t in thresholds: 
    print(f"Measuring performance for similarity threshold {t}...")
    query_times = []
    for rdmol in first_200:
        temp_times = []
        for r in range(repetitions):
            start = time.time()
            _ = similarity.SimSearchLSH(rdmol, db, db.molecules_100K, 
                                        db.molecules_100KP, db.molecules_100KCt, threshold=t)
            end = time.time()
            temp_times.append(end - start)
        query_times.append(mean(temp_times))
    LSH_means_100K.append([t, mean(query_times)])
    LSH_medians_100K.append([t, median(query_times)])

print(f"LSH means: {LSH_means_100K}")
print(f"LSH medians: {LSH_medians_100K}")

x_list = [v[0] for v in LSH_medians_100K]
y_list = [v[1] for v in LSH_medians_100K]
plt.xlabel('thresholds')
plt.ylabel('time (s)')
plt.title('SimSearchLSH medians / 100K dataset')
plt.plot(x_list, y_list)

In [ ]:
# Benchmark against the first 100,000 molecules in ChEMBL. 
LSH_means_1M = []
LSH_medians_1M = []

for t in thresholds: 
    print(f"Measuring performance for similarity threshold {t}...")
    query_times = []
    for rdmol in first_200:
        temp_times = []
        for r in range(repetitions):
            start = time.time()
            _ = similarity.SimSearchLSH(rdmol, db, db.molecules_1M, 
                                        db.molecules_1MP, db.molecules_1MCt, threshold=t)
            end = time.time()
            temp_times.append(end - start)
        query_times.append(mean(temp_times))
    LSH_means_1M.append([t, mean(query_times)])
    LSH_medians_1M.append([t, median(query_times)])

print(f"LSH means: {LSH_means_1M}")
print(f"LSH medians: {LSH_medians_1M}")

x_list = [v[0] for v in LSH_medians_1M]
y_list = [v[1] for v in LSH_medians_1M]
plt.xlabel('thresholds')
plt.ylabel('time (s)')
plt.title('SimSearchLSH medians / 1M dataset')
plt.plot(x_list, y_list)

In order to compare accuracy, we will use the approach written about in the ChEMBL blog post: finding the symmetric set difference between the two sets of results as a percentage of the size of the union of the two result sets. 

In [ ]:
results = []

for t in thresholds: 
    print(f"Measuring accuracy for similarity threshold {t}...")
    nmols_w_discrepancies = 0
    discrepancies_per_mol = []
    discrepancy_percent_per_mol = []
    for rdmol in first_200:
        sim_lsh = similarity.SimSearchLSH(rdmol, db, db.molecules_100K, 
                                          db.molecules_100KP, db.molecules_100KCt, threshold=t)
        sim_agg = similarity.SimSearchAggregate(rdmol, db.molecules_100K, db.molecules_100KCt, threshold=t)
        if sim_lsh: 
            set_lsh = set(result[1] for result in sim_lsh)
        else:
            set_lsh = set()
        if sim_agg: 
            set_agg = set(result[1] for result in sim_agg)
        else: 
            set_agg = set()
        sym_set_diff = (set_lsh ^ set_agg)
        discrepancies = len(sym_set_diff)
        total = len(set_lsh | set_agg)
        if discrepancies:
            nmols_w_discrepancies += 1
            discrepancies_per_mol.append(discrepancies)
            discrepancy_percent_per_mol.append(discrepancies / total * 100)
    results.append([t, f'nmols_w_discrepancies: {nmols_w_discrepancies}', 
                    np.mean(discrepancies_per_mol), np.mean(discrepancy_percent_per_mol)])
print(results)
x_list = [v[0] for v in results]
y_list = [v[3] for v in results]
plt.xlabel('thresholds')
plt.ylabel('discrepancy percent per molecule')
plt.title('LSH Accuracy / 100K dataset')
plt.plot(x_list, y_list)

### Discussion
These times are already very reasonable for a similarity search. However, it is worth noting that these benchmarks were run on a local MongoDB instance, effectively making no distinction between the client and the server. A MongoDB instance that has more horizontal scaling could benefit greatly from the aggregation pipeline, thus speeding search even further. 

The time complexity also increases greatly with decreasing Tanimoto thresholds.